In [ ]:
%%capture
!pip install --no-index --find-links ../input/ktrain/ ../input/ktrain/setuptools_scm-6.0.1-py3-none-any.whl
!pip install --no-index --find-links ../input/ktrain/ ../input/ktrain/ktrain-0.26.4.tar.gz 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import mean_squared_error

import ktrain
from ktrain import text

In [ ]:
data=pd.read_csv('../input/commonlitreadabilityprize/train.csv')
data=data[['excerpt','target']]
data.head()

In [ ]:
test=pd.read_csv('../input/commonlitreadabilityprize/test.csv')
test.head()

# transformer

In [ ]:
def transformer(MODEL_NAME):
    preproc  = text.Transformer(MODEL_NAME, maxlen=200)
    x_test=test.excerpt.values
    x_test = preproc.preprocess_test(x_test)
    predictions,rmses=[],[]
    for train_index, val_index in KFold(5).split(data):
        train,val=data.iloc[train_index],data.iloc[val_index]
        x_train=train.excerpt.values
        x_val=val.excerpt.values

        y_train=train.target.values
        y_val=val.target.values

        trn = preproc.preprocess_train(x_train, y_train)
        val = preproc.preprocess_test(x_val, y_val)
        model = preproc.get_regression_model()
        learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=24)
        learner.fit_onecycle(1e-5, 5)
        print('mse',learner.validate())
        pred=learner.predict(val)
        rmse=mean_squared_error(y_val,pred,squared=False)
        print('rmse',rmse)
        predictions.append(learner.predict(x_test))
        rmses.append(rmse)
    return predictions,rmses

In [ ]:
%%capture
MODEL_NAME_0 = '../input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased'
predictions0,rmses0=transformer(MODEL_NAME_0)
mean0=np.mean(rmses0)

In [ ]:
%%capture
MODEL_NAME_1 = '../input/huggingface-bert-variants/distilbert-base-uncased/distilbert-base-uncased'
predictions1,rmses1=transformer(MODEL_NAME_1)
mean1=np.mean(rmses1)


In [ ]:
%%capture
MODEL_NAME_2 = '../input/huggingface-roberta-variants/distilroberta-base/distilroberta-base'
predictions2,rmses2=transformer(MODEL_NAME_2)
mean2=np.mean(rmses2)


In [ ]:
%%capture
MODEL_NAME_3 = '../input/huggingface-roberta-variants/roberta-base/roberta-base'
predictions3,rmses3=transformer(MODEL_NAME_3)
mean3=np.mean(rmses3)


In [ ]:
print('bert mean',rmses0,'all',mean0)
print('dbert mean',rmses1,'all',mean1)
print('droberta mean',rmses2,'all',mean2)
print('roberta mean',rmses3,'all',mean3)

In [ ]:
pred0=np.mean(predictions0,0).ravel()
pred1=np.mean(predictions1,0).ravel()
pred2=np.mean(predictions2,0).ravel()
pred3=np.mean(predictions3,0).ravel()
pred1.shape

In [ ]:
pred=np.average([pred0,pred1,pred2,pred3],weights=[1-mean0,1-mean1,1-mean2,1-mean3],axis=0)
pred.shape


In [ ]:
sub=pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
sub.target=pred
sub.to_csv('submission.csv',index=False)
sub.head()